In [1]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [3]:
net = models.__dict__['resnet18']()

In [6]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
ly = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(512, 1)
)   

net.fc = ly

In [8]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class smallNet(nn.Module):

    def __init__(self, dropout=.5):
        super(smallNet, self).__init__()
        self.seq1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),    
            nn.MaxPool2d(2)
        )  
        
        self.seq2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )          
              
        # fc
        self.fc = nn.Linear(16384, 1)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight.data)
                nn.init.constant_(m.bias.data,0.1)


        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
#         print('input:', x.size())
        x = self.seq1(x)
        x = self.seq2(x)
        x = self.seq2(x)
        x = self.seq2(x)

        
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


In [14]:
s = smallNet()
s

smallNet(
  (seq1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (seq2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=16384, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
? nn.Conv2d

In [38]:
class smallNet(nn.Module):

    def __init__(self):
        super(smallNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv = nn.Conv2d(3, 64, 3)
        # batch         
        self.batch = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        # relu
        self.relu = nn.ReLU()
        # max pooling        
        self.maxpool = nn.MaxPool2d(2)
#         # full connected         
#         self.fc = nn.Linear(14400, 1)

    def forward(self, x):
        print(x.size())
        x = self.conv(x)
        print(x.size())
        x = self.batch(x)
        print(x.size())
        x = self.relu(x)
        print(x.size())
        x = self.maxpool(x)
        print(x.size())
        x = x.view(-1, self.num_flat_features(x))
        print(x.size())
        return nn.Linear(x.size()[1], 1)(x)
#         # Max pooling over a (2, 2) window
#         x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
#         # If the size is a square you can only specify a single number
#         x = F.max_pool2d(F.relu(self.conv2(x)), 2)
#         x = x.view(-1, self.num_flat_features(x))
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = smallNet()
print(net)

smallNet(
  (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (batch): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [39]:
params = list(net.parameters())
for param in params:
    print(param.size())

torch.Size([64, 3, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])


In [43]:
input = torch.randn(3, 3, 32, 32)
out = net(input)
print(out)

torch.Size([3, 3, 32, 32])
torch.Size([3, 64, 30, 30])
torch.Size([3, 64, 30, 30])
torch.Size([3, 64, 30, 30])
torch.Size([3, 64, 15, 15])
torch.Size([3, 14400])
tensor([[0.0154],
        [0.0214],
        [0.2525]], grad_fn=<AddmmBackward>)


In [5]:
?nn.Dropout

Init signature: nn.Dropout(p=0.5, inplace=False)
Docstring:     
During training, randomly zeroes some of the elements of the input
tensor with probability :attr:`p` using samples from a Bernoulli
distribution. Each channel will be zeroed out independently on every forward
call.

This has proven to be an effective technique for regularization and
preventing the co-adaptation of neurons as described in the paper
`Improving neural networks by preventing co-adaptation of feature
detectors`_ .

Furthermore, the outputs are scaled by a factor of :math:`\frac{1}{1-p}` during
training. This means that during evaluation the module simply computes an
identity function.

Args:
    p: probability of an element to be zeroed. Default: 0.5
    inplace: If set to ``True``, will do this operation in-place. Default: ``False``

Shape:
    - Input: :math:`(*)`. Input can be of any shape
    - Output: :math:`(*)`. Output is of the same shape as input

Examples::

    >>> m = nn.Dropout(p=0.2)
    >>> inpu

In [1]:
from datetime import datetime,timezone,timedelta
dt = datetime.utcnow()
print(dt)
dt = dt.replace(tzinfo=timezone.utc)
print(dt)
tzutc_8 = timezone(timedelta(hours=8))
local_dt = dt.astimezone(tzutc_8)
print(local_dt)

2020-02-18 10:13:31.992813
2020-02-18 10:13:31.992813+00:00
2020-02-18 18:13:31.992813+08:00


In [2]:
type(local_dt)

datetime.datetime

In [4]:
print(str(local_dt))

2020-02-18 18:13:31.992813+08:00


In [4]:
checkpoint = torch.load('checkpoints/checkpoints_pure/checkpoint-2020-02-20 13:38:29.968086+08:00/312.pth.tar', map_location='cpu')

In [ ]:
checkpoint

In [ ]:
print(checkpoint)